In [1]:
import numpy as np
import astropy.io.fits as fits
from astropy.table import Table
from astromodels import Powerlaw,  PointSource, SpectralComponent
import astropy.time as at
from datetime import datetime
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets
import os

[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)
[WARNING ] ROOT minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] No fermitools installed
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances 

In [2]:
data_path = "crab_data/0374"

In [3]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])

In [4]:
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])

In [5]:
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])

In [6]:
# Define Source and Spectrum
ra, dec = 10, -40
K, piv, index = 8e-3, 40, -2
    
pl = Powerlaw()
pl.piv = piv
pl.K = K
pl.index = index
component1 = SpectralComponent("pl", shape=pl)
source = PointSource("Test", ra=ra, dec=dec, components=[component1])

emod = np.geomspace(10, 3000, 50)
spec = source(emod)
spec_binned = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2

In [7]:
# Generate Source Counts

assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)

source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)

for p_i, pointing in enumerate(pointings):
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], ra, dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mat = sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1))
    
    count_rates = np.dot(spec_binned, resp_mat)
    
    for d_i, d in enumerate(dets):
        index = p_i * 85 + d
        source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[index])    


Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [8]:
# Save Data for PySpi

# source_counts = np.array(source_counts, dtype=np.uint32)

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
    counts = t
    
updated_counts = counts.copy()
updated_counts["COUNTS"] += source_counts

hdu = fits.BinTableHDU(data=updated_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{data_path}/evts_det_spec.fits")

OSError: File crab_data/0374/evts_det_spec.fits already exists. If you mean to replace it then use the argument "overwrite=True".

In [13]:
spimodfit_folder = "crab_data/0374_spimodfit"

if not os.path.exists(spimodfit_folder):
    os.mkdir(spimodfit_folder)

spimodfit_pointings = ["03740001",
"03740002",
"03740003",
"03740004",
"03740005",
"03740006",
"03740007",
"03740008",
"03740009",
"03740010",
"03740011",
"03740012",
"03740013",
"03740014",
"03740015",
"03740016",
"03740017",
"03740018",
"03740019",
"03740020",
"03740021",
"03740022",
"03740023",
"03740024",
"03740025",
"03740026",
"03740027",
"03740028",
"03740029",
"03740030",
"03740031",
"03740032",
"03740033",
"03740035",
"03740036",
"03740037",
"03740038",
"03740039",
"03740040",
"03740041",
"03740042",
"03740043",
"03740045",]

spimodfit_energy_bins = [20.0, 21.5, 23.5, 25.5, 27.5, 30.0, 32.5, 35.5, 38.5, 42.0, 45.5, 49.5, 54.0, 58.5, 63.5, 69.0, 75.0, 81.5, 89.0, 96.5, 105.0, 114.0, 124.0, 134.5, 146.0, 159.0, 172.5, 187.5, 204.0, 221.5, 240.5, 261.5, 284.0, 308.5, 335.5, 364.5, 396.0, 430.0, 467.5, 508.0, 514.0, 600.0,]

In [31]:
# Pointing Indices
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    p_indices = []
    for p in spimodfit_pointings:
        temp = np.argwhere(t["PTID_ISOC"]==p)
        if len(temp)>0:
            p_indices.append(temp[0][0])

# Energy Indices
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    e_indices = []
    for e in spimodfit_energy_bins:
        temp = np.argwhere(t["E_MIN"]==e)
        if len(temp)>0:
            e_indices.append(temp[0][0])
            
# Single Event Indices
se_indices = []
for i in p_indices:
    for j in range(19):
        se_indices.append(i*85 + j)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 42]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 765, 766, 767, 768, 769, 770, 771

In [79]:
# energy_boundaries.fits.gz
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])

eb = t[e_indices[:-1]]
eb["E_MAX"] = t["E_MIN"][e_indices[1:]]

hdu = fits.BinTableHDU(data=eb, name="SPI.-EBDS-SET")
hdu.writeto(f"{spimodfit_folder}/energy_boundaries.fits.gz")

In [33]:
# dead_time.fits.gz
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    
dt = t[se_indices]

hdu = fits.BinTableHDU(data=dt, name="SPI.-OBS.-DTI")
hdu.writeto(f"{spimodfit_folder}/dead_time.fits.gz")

In [36]:
# pointing.fits.gz
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
ps = t[p_indices]

hdu = fits.BinTableHDU(data=ps, name="SPI.-OBS.-PNT")
hdu.writeto(f"{spimodfit_folder}/pointing.fits.gz")

In [65]:
# evts_det_spec.fits.gz
with fits.open(f"{data_path}/evts_det_spec.fits") as file:
    t = Table.read(file[1])
    
eds_temp = t[se_indices]
eds = eds_temp.copy()
eds["COUNTS"] = eds["COUNTS"][:,e_indices[:-1]]

for i in range(len(e_indices)-1):
    eds["COUNTS"][:,i] = np.sum(eds_temp["COUNTS"][ : , e_indices[i] : e_indices[i+1]], axis=1)
    
hdu = fits.BinTableHDU(data=eds, name="SPI.-OBS.-DSP")
hdu.writeto(f"{spimodfit_folder}/evts_det_spec.fits.gz")

In [82]:
# gti.fits.gz
with fits.open(f"{data_path}/gti.fits") as file:
    t = Table.read(file[1])

gt = t[se_indices]

hdu = fits.BinTableHDU(data=gt, name="SPI.-OBS.-GTI")
hdu.writeto(f"{spimodfit_folder}/gti.fits.gz")

In [78]:
# simulated source
with fits.open(f"{data_path}/cat_crab.fits.gz") as file:
    t = Table.read(file[1])
    
ns = t.copy()
ns["RA_OBJ"] = 10.
ns["DEC_OBJ"] = -40.
ns["NAME"] = "SimSource"
ns["CLASS"] = 0
ns["ISGRI_FLAG"] = 1
ns["SPI_FLUX_1"] = 0.
ns["SPI_FLUX_2"] = 0.
ns["ISGR_FLUX_1"] = 0.
ns["ISGR_FLUX_2"] = 0.
ns["PICS_FLUX_1"] = 0.
ns["PICS_FLUX_2"] = 0.
ns["JEMX_FLUX_1"] = 0.
ns["JEMX_FLUX_2"] = 0.
ns["BIRD_SOURCE_ID"] = "J000000.0+000000"
ns["SOURCE_ID"] = "SimSource"

hdu = fits.BinTableHDU(data=ns, name="SPI.-SRCL-CAT")
hdu.writeto(f"{spimodfit_folder}/cat_sim.fits.gz")
